In [2]:
#iteration 1 - testing models on feature engineered dataset
#calculating a Dummy classfier - baseline for accuracy

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Print column names to inspect
it3_df = pd.read_excel('SpotifyDataIt3.xlsx')

In [3]:
print(it3_df.columns)

Index(['Unnamed: 0', 'artists_num', 'album_num_tracks', 'peak_rank',
       'weeks_on_chart', 'streams', 'danceability', 'energy', 'key',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration', 'release_year', 'release_month',
       'release_day', 'release_dayofweek', 'loudness_log',
       'energy_danceability_interaction'],
      dtype='object')


In [4]:
# Separate features (X) and labels (y)
X = it3_df[['artists_num', 'album_num_tracks', 'peak_rank','weeks_on_chart','danceability', 'energy', 'key', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration','release_year',
       'release_month', 'release_day', 'release_dayofweek', 'loudness_log','energy_danceability_interaction']]
y = it3_df['streams']  

# Split the dataset into 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Display the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (18088, 20)
X_test shape: (7752, 20)
y_train shape: (18088,)
y_test shape: (7752,)


In [5]:
#Importing necessary libraries and loading the dataset
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the Linear Regression model
linear_model = LinearRegression()

# Training the model
linear_model.fit(X_train, y_train)

# Predicting the test set results
y_pred_linear = linear_model.predict(X_test)

# Evaluating the model
mse_linear = mean_squared_error(y_test, y_pred_linear)
rmse_linear = np.sqrt(mse_linear)
r2_linear = r2_score(y_test, y_pred_linear)

# Returning the evaluation metrics
mse_linear, rmse_linear, r2_linear

(3650387003708634.5, 60418432.64856044, 0.08279784727712913)

First of we cross validate the linear regression model to evalute the models performance

In [6]:
from sklearn.model_selection import cross_val_score

# Initialize the Linear Regression model
linear_model = LinearRegression()

# Perform cross-validation
cv_scores = cross_val_score(linear_model, X, y, cv=5, scoring='neg_mean_squared_error')

# Convert MSE scores to RMSE scores
rmse_scores = np.sqrt(-cv_scores)

# Output the cross-validation RMSE scores
rmse_scores


array([61466224.18909626, 62199008.53080785, 59645209.04620065,
       62158663.4592537 , 62029685.81215119])

In [7]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LassoCV

# Start by finding the best alpha for the L1 regression to optimize the model
# Setting up a range of alpha values to test
alphas = np.logspace(-4, 0.01, 10)

# Initialize and train the LassoCV model to find the best alpha
lasso_cv_model = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_cv_model.fit(X_train, y_train)

# Optimal alpha found by LassoCV
optimal_alpha = lasso_cv_model.alpha_

# Initialize the Lasso regression model (L1 regularization)
# Setting the alpha to the optimal alpha found above
lasso_model = Lasso(alpha=optimal_alpha, random_state=30)

# Training the Lasso regression model
lasso_model.fit(X_train, y_train)

# Predicting the test set results
y_pred_lasso = lasso_model.predict(X_test)

# Evaluating the model
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f'MSE (Lasso): {mse_lasso}, RMSE (Lasso): {rmse_lasso}, R-squared (Lasso): {r2_lasso}')

MSE (Lasso): 3650387071617491.0, RMSE (Lasso): 60418433.21054834, R-squared (Lasso): 0.08279783021423981


To try further explore the model and make it better we implement a GridSearch for the Lasso regression model below:

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

# Assuming X_train, y_train, X_test, y_test are already defined
# Define the parameter grid for GridSearch
param_grid = {
    'alpha': [0.1, 1, 10, 100],
    'max_iter': [1000, 5000, 10000],
    'tol': [0.0001, 0.001, 0.01]
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=Lasso(random_state=42), param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters
best_parameters = grid_search.best_params_

# Output the best parameters
best_parameters



{'alpha': 100, 'max_iter': 1000, 'tol': 0.0001}

Here we use the "best parameters" to see the change in MSE, RMSE and R-squared

In [9]:
# Initialize the Lasso regression model with the best parameters
best_lasso_model = Lasso(**best_parameters, random_state=42)

# Training the Lasso regression model with the best parameters
best_lasso_model.fit(X_train, y_train)

# Predicting the test set results using the best model
y_pred_best_lasso = best_lasso_model.predict(X_test)

# Evaluating the best model
mse_best_lasso = mean_squared_error(y_test, y_pred_best_lasso)
rmse_best_lasso = np.sqrt(mse_best_lasso)
r2_best_lasso = r2_score(y_test, y_pred_best_lasso)

# Printing the evaluation metrics for the best model
print(f'MSE (Best Lasso): {mse_best_lasso}, RMSE (Best Lasso): {rmse_best_lasso}, R-squared (Best Lasso): {r2_best_lasso}')

MSE (Best Lasso): 3650393662497363.5, RMSE (Best Lasso): 60418487.75414164, R-squared (Best Lasso): 0.08279617417908491


In [30]:
#Here we create a gradient tree boosting regressor model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize and train the Gradient Boosting Regressor
gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbr_model.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred_gbr = gbr_model.predict(X_test)
mse_gbr = mean_squared_error(y_test, y_pred_gbr)
rmse_gbr = np.sqrt(mse_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)

mse_gbr, rmse_gbr, r2_gbr
print(f'MSE (gbr): {mse_gbr}, RMSE (gbr): {rmse_gbr}, R-squared (gbr): {r2_gbr}')

MSE (gbr): 2871076330363802.5, RMSE (gbr): 53582425.57372522, R-squared (gbr): 0.27860870966119944


Now we implement random search for Gradient Booster Regressor

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform, randint
import numpy as np

# Assuming X_train, y_train, X_test, y_test are already defined

# Define the parameter distributions for RandomizedSearchCV
param_distributions = {
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(2, 6),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'subsample': uniform(0.8, 0.2)
}

# Create RandomizedSearchCV object for GradientBoostingRegressor
random_search_gbr = RandomizedSearchCV(estimator=GradientBoostingRegressor(random_state=42), 
                                       param_distributions=param_distributions, 
                                       n_iter=10, 
                                       scoring='neg_mean_squared_error', 
                                       cv=5, 
                                       random_state=42,
                                       verbose=1)

# Fit RandomizedSearchCV
random_search_gbr.fit(X_train, y_train)

# Best parameters
best_parameters_gbr = random_search_gbr.best_params_

# Using the best parameters to train and evaluate the model
best_gbr_model = GradientBoostingRegressor(**best_parameters_gbr, random_state=42)
best_gbr_model.fit(X_train, y_train)

# Predicting and evaluating the best model
y_pred_best_gbr = best_gbr_model.predict(X_test)
mse_best_gbr = mean_squared_error(y_test, y_pred_best_gbr)
rmse_best_gbr = np.sqrt(mse_best_gbr)
r2_best_gbr = r2_score(y_test, y_pred_best_gbr)

# Print the evaluation metrics for the best model
print(f'MSE (Best GBR): {mse_best_gbr}, RMSE (Best GBR): {rmse_best_gbr}, R-squared (Best GBR): {r2_best_gbr}')
print(f'Best Parameters: {best_parameters_gbr}')



Fitting 5 folds for each of 10 candidates, totalling 50 fits
MSE (Best GBR): 2590181037793240.0, RMSE (Best GBR): 50893821.21430106, R-squared (Best GBR): 0.3491869159647206
Best Parameters: {'learning_rate': 0.13022300234864176, 'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 102, 'subsample': 0.9939819704323989}


In [31]:
#Implementing a XGBoost model
import xgboost as xgb

# Initialize and train the XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_model.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

mse_xgb, rmse_xgb, r2_xgb

(2836900423627351.0, 53262561.18163443, 0.287195803357863)

Here we use gridsearch to further investigate and tune the XGBoost model

In [42]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define a range of hyperparameters for GridSearch
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Create GridSearchCV object for XGBRegressor
grid_search_xgb = GridSearchCV(estimator=XGBRegressor(random_state=42), 
                               param_grid=param_grid, 
                               scoring='neg_mean_squared_error', 
                               cv=5, 
                               verbose=1)

# Fit GridSearchCV
grid_search_xgb.fit(X_train, y_train)

# Best parameters
best_parameters_xgb = grid_search_xgb.best_params_

# Output the best parameters
best_parameters_xgb

# Using the best parameters to train and evaluate the model
best_xgb_model = XGBRegressor(**best_parameters_xgb, random_state=42)
best_xgb_model.fit(X_train, y_train)

# Predicting and evaluating the best model
y_pred_best_xgb = best_xgb_model.predict(X_test)
mse_best_xgb = mean_squared_error(y_test, y_pred_best_xgb)
rmse_best_xgb = np.sqrt(mse_best_xgb)
r2_best_xgb = r2_score(y_test, y_pred_best_xgb)

# Print the evaluation metrics for the best model
mse_best_xgb, rmse_best_xgb, r2_best_xgb

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


(2534497509003570.5, 50343793.15271715, 0.3631780496240292)